In [ ]:
import torch
from foolbox.attacks import PGD
from foolbox.distances import Linfinity
from foolbox.models import PyTorchModel
from ibp_neurips19.models import small_cnn
from ibp_neurips19.datasets import get_dataset
from torch.utils.data.dataset import random_split

In [ ]:
net = small_cnn().eval()
checkpoint = torch.load('checkpoint.pth')
net.load_state_dict(checkpoint['state_dict'])
dataset, _ = random_split(get_dataset('MNIST', False), [int(1e2), int(1e4-1e2)])
normalization = dataset.dataset.transform.transforms[1]
net = PyTorchModel(
    net,
    bounds=(0, 1),
    num_classes=len(dataset.dataset.classes),
    preprocessing=(normalization.mean, normalization.std),
    device='cpu',
)
dataset.dataset.transform = dataset.dataset.transform.transforms[0]
image, label = dataset[0]

In [ ]:
kwargs = dict(unpack=False, binary_search=False, epsilon=0.12, stepsize=0.1, iterations=20, random_start=True)
attack = PGD(net, distance=Linfinity)
adversarial = attack(image.numpy(), int(label), **kwargs)
failed = (adversarial.distance.value == 0 or
          adversarial.reached_threshold() or
          adversarial.image is None)
adversarial.distance.value